In [10]:
import numpy as np
from osgeo import gdal, ogr
from sklearn import metrics
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

# Load Validation data and convert it to raster
naip_fn = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/verschiedene Stacks/Stack_rgb_dem_ndvi_nir.tif"

driverTiff = gdal.GetDriverByName('GTiff')
naip_ds = gdal.Open(naip_fn)

test_fn = "D:/cbr/VP2/VP2_SoilMapping/Shapes/test_klassiert.shp"
test_ds = ogr.Open(test_fn)
lyr = test_ds.GetLayer()
driver = gdal.GetDriverByName('MEM')
target_ds = driver.Create('', naip_ds.RasterXSize, naip_ds.RasterYSize, 1, gdal.GDT_UInt16)
target_ds.SetGeoTransform(naip_ds.GetGeoTransform())
target_ds.SetProjection(naip_ds.GetProjection())
options = ['ATTRIBUTE=id']
gdal.RasterizeLayer(target_ds, [1], lyr, options=options)

# load the predicted classified image
truth = target_ds.GetRasterBand(1).ReadAsArray()
pred_ds = gdal.Open('D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/verschiedene Stacks/Klassifikation_rgb_dem_ndvi_nir_10000_fs.tif')
pred = pred_ds.GetRasterBand(1).ReadAsArray()

idx = np.nonzero(truth)

# create confusion matrix from the truth (validation) data and the predicted data
cm = metrics.confusion_matrix(truth[idx], pred[idx])

#calculate Indices
kappa = metrics.cohen_kappa_score(truth[idx], pred[idx])
precision = metrics.precision_score(truth[idx], pred[idx], average='weighted')
recall = metrics.recall_score(truth[idx], pred[idx], average='weighted')
accuracy = metrics.accuracy_score(truth[idx], pred[idx])
f1 = metrics.f1_score(truth[idx], pred[idx], average='weighted')

# Make the confusion matrix more beautiful
corr_matrix = pd.DataFrame(data=cm, index=["Grass", "Other", "Pead", "Possible Peat"], columns=["Prd. Grass", "Prd. Other", "Prd. Peat", "Pred. Possib. Peat"])
print(corr_matrix)

print(" ")
print("____ Statistics ____")
print("Kappa     :", str(round(kappa * 100, 2)),  "%")
print("Precision :", str(round(precision * 100, 2)), "%")
print("Recall    :", str(round(recall * 100, 2)), "%")
print("Accuracy  :", str(round(accuracy * 100, 2)), "%")
print("F1        :", str(round(f1 * 100, 2)), "%")

# Old indices. Not longer in use because they work only for two classes, but still good to know
#A = cm[0][0]
#B = cm[0][1]
#C = cm[1][0]
#D = cm[1][1]
#accuracy = cm.diagonal() / cm.sum(axis=0)
#print("Accuracy :                   ", accuracy)
#FAR = (cm[0][1]) / ((cm[0][1]) + (cm[0][0]))
#print("False Alarm Ratio :          ", FAR)



               Prd. Grass  Prd. Other  Prd. Peat  Pred. Possib. Peat
Grass                   5           2          0                   0
Other                   0          16          0                   0
Pead                    0           0          8                   0
Possible Peat           1           3          0                   0
 
____ Statistics ____
Kappa     : 73.18 %
Precision : 74.35 %
Recall    : 82.86 %
Accuracy  : 82.86 %
F1        : 77.78 %
